In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2019-20.csv"
relative_string_map = "./Data_Goal/String_map.csv"
relative_semantic = "./Data_Goal/Semantic_map.csv"
relative_sem_threshold = "./Data_Goal/Semantic_threshold_map.csv"
relative_rank = "./Data_Goal/Ranking.csv"
relative_rank_unroll = "./Data_Goal/Ranking_unroll.csv"
relative_tab_filter = "./Data_Goal/Ranking_tab_filter.csv"
relative_chap_filter = "./Data_Goal/Ranking_chap_filter.csv"
relative_final_map = "./Mapping_Goal_2019-20.csv"
relative_count = "./Data_Goal/Comparison_count.csv"


# Get the absolute path by joining the current directory with the relative path
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_rank =os.path.normpath( os.path.join(os.getcwd(), relative_rank))
absolute_path_rank_unroll =os.path.normpath( os.path.join(os.getcwd(), relative_rank_unroll))
absolute_path_tab_filter =os.path.normpath( os.path.join(os.getcwd(), relative_tab_filter))
absolute_path_chap_filter =os.path.normpath( os.path.join(os.getcwd(), relative_chap_filter))
absolute_path_final_map =os.path.normpath( os.path.join(os.getcwd(), relative_final_map))
absolute_path_count =os.path.normpath( os.path.join(os.getcwd(), relative_count))

In [3]:
def process_result(result_sdg, sdg_df, chapter_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['Rank']
        goal_no = row['Goal No.']
        
        for attr in attr_id:
            rows.append({
                'Attr_id': attr,
                'top_n_goal_id': goal_no
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    chapter_id = dict(zip(chapter_df['Attr_id'], chapter_df['Chapter_id']))
    chapter_name = dict(zip(chapter_df['Chapter_id'], chapter_df['Chapter_name']))
    table_id = dict(zip(chapter_df['Attr_id'], chapter_df['Table_id']))
    table_name = dict(zip(chapter_df['Table_id'], chapter_df['Table_name']))
    description = dict(zip(chapter_df['Attr_id'], chapter_df['Description']))
    all_goals['Chapter_id'] = all_goals['Attr_id'].map(chapter_id.get)
    all_goals['Chapter_name'] = all_goals['Chapter_id'].map(chapter_name.get)
    all_goals['Table_id'] = all_goals['Attr_id'].map(table_id.get)
    all_goals['Table_name'] = all_goals['Table_id'].map(table_name.get)
    all_goals['Description'] = all_goals['Attr_id'].map(description.get) 
    
                
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'top_n_goal_id']]

    return all_goals


In [4]:
# Function to combine and remove duplicates from lists while maintaining order
def combine_lists_BC(row):
    list_B = row['top_n_attr_B']
    list_C = row['top_n_attr_C']
    
    # Combine lists and remove duplicates while maintaining order
    items = list(OrderedDict.fromkeys(list_B + list_C))
    return items

In [5]:
# Function to find intersection while maintaining order
def find_intersection_ABC(row):
    intersection = [x for x in row['top_n_attr_A'] if x in row['B_plus_C']]
    return intersection

In [6]:
def find_difference_A(row):
    top_n_attr = row['top_n_attr_A']
    intersection = row['A_and_BC']
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [7]:
# Function to find intersection while maintaining order
def find_intersection_BC(row):
    intersection = [x for x in row['top_n_attr_B'] if x in row['top_n_attr_C']]
    return intersection

In [8]:
def find_difference_B(row):
    top_n_attr = row['top_n_attr_B']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [9]:
def find_difference_C(row):
    top_n_attr = row['top_n_attr_C']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'] + row['B_minus_all'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [10]:
def final_rank(row):
    #combined_top_n_attr = sorted(set(list_B + list_C), key=lambda x: (list_B + list_C).index(x))
    items = row['A_and_BC'] + row['A'] + row['B_and_C'] + row['B_minus_all'] + row['C_minus_all']
    item_set = list(OrderedDict.fromkeys(items))    
    return item_set

In [11]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [12]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [13]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id', 'Targets', 'Target_id'],axis=1,inplace=True)

In [14]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [15]:
sdg.reset_index(inplace = True, drop = True)

In [16]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [17]:
data_new = data.copy()
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Taluks
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Hoblies
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Grama Panchayaths
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Circles
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages"
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019"
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019"
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019"
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019"


In [18]:
sdg_string = pd.read_csv(absolute_path_string, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_string

,Goal No.,Goal,new_goal,new_goal_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"['end', 'poverty', 'form', 'everywhere']",end poverty in all its forms everywhere,250,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, ...",['Ration Shops and Ration Card (Below poverty ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['General Information', 'General Information',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Urban,Ration shops, Ration Shops and Ration ..."
1,2,"End hunger, achieve food security and improved...","['end', 'hunger', 'achieve', 'food', 'security...",end hunger achieve food security and improved ...,500,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[4.3, 4.3, 4.6, 4.6, 4.12, 4.1, 4.1, 4.1, 4.1,...",['Area Under Principal crops in Hectares 2018-...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","['Agriculture, Horticulture & Sericulture', 'A...","[2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Total Food Grains,Pulses, Area Under Princip..."
2,3,Ensure healthy lives and promote well-being fo...,"['ensure', 'healthy', 'life', 'promote', 'well...",ensure healthy lives and promote well being fo...,36,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",['Age wise Population per 2011 Census and Proj...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Area and Population', 'Area and Population',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Male,0-14 2011, Age wise Population per 2011..."
3,4,Ensure inclusive and equitable quality educati...,"['ensure', 'inclusive', 'equitable', 'quality'...",ensure inclusive and equitable quality educati...,128,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10....","['Literates 2011 Census', 'Literates 2011 Cens...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['Education', 'Education', 'Education', 'Educa...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Male,Rural, Literates 2011 Census, Education..."
4,5,Achieve gender equality and empower all women ...,"['achieve', 'gender', 'equality', 'empower', '...",achieve gender equality and empower all women ...,96,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[13.5, 10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10....",['No. of Anganawadi Teachers Workers and Suppl...,"[13, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","['Women & Child Development', 'Education', 'Ed...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Adolescent Girls,No. of Beneficiaries under ..."
5,6,Ensure availability and sustainable management...,"['ensure', 'availability', 'sustainable', 'man...",ensure availability and sustainable management...,30,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","['No. of Fire Stations on 31-3-2020', 'Length ...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","['General Information', 'Transport and Communi...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Working Fire Brigade Water Tankers, No. of F..."
6,7,"Ensure access to affordable, reliable, sustain...","['ensure', 'access', 'affordable', 'reliable',...",ensure access to affordable reliable sustainab...,51,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 15.2, 16.1...",['Length of Rural Roads and Number of Habitati...,"[9, 9, 9, 9, 9, 9, 9, 15, 16, 16, 16, 16, 16, ...","['Transport and Communication', 'Transport and...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Pucca Road,Panchayat Roads,Length of Rural R..."
7,8,"Promote sustained, inclusive and sustainable e...","['promote', 'sustained', 'inclusive', 'sustain...",promote sustained inclusive and sustainable ec...,17,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380

In [19]:
sdg_sim = pd.read_csv(absolute_path_semantic, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_sim

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"['end', 'poverty', 'form', 'everywhere']",end poverty in all its forms everywhere,"[['end', 'poverty'], ['end'], ['forms', 'every...",[[ 0.004565 0.380225 0.28172 ... -0.0...,"[1366, 1367, 1368, 1369, 1370, 1371, 1372, 21,...","[0.8735124280076773, 0.5514384505494149, 0.512...",40,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[14, 14, 14, 14, 14, 14, 14, 1, 1, 1, 1, 1, 1,...","[14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 1.6...","['Ddu-gky,Nrlm National Rural Livelihood Missi...","['Rural Development and Panchayat Raj', 'Rural...",['Skill Development and Self Employment at the...
1,2,"End hunger, achieve food security and improved...","['end', 'hunger', 'achieve', 'food', 'security...",end hunger achieve food security and improved ...,"[['sustainable', 'agriculture'], ['security', ...",[[ 0.079023 0.0409187 -0.25506 ... -0.3...,"[380, 381, 455, 456, 510, 858, 859, 860, 861, ...","[0.8411997113893511, 0.8556416402619208, 0.566...",60,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[4, 4, 4, 4, 4, 6, 6, 6, 6, 17, 17, 17, 17, 17...","[4.3, 4.3, 4.6, 4.6, 4.12, 6.2, 6.2, 6.2, 6.2,...","['Total Food Grains,Pulses, Area Under Princip...","['Agriculture, Horticulture & Sericulture', 'A...",['Area Under Principal crops in Hectares 2018-...
2,3,Ensure healthy lives and promote well-being fo...,"['ensure', 'healthy', 'life', 'promote', 'well...",ensure healthy lives and promote well being fo...,"[['well'], ['well', 'ages'], ['ensure'], ['pro...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[352, 353, 354, 1393, 1394, 1414, 967, 968, 96...","[0.7045711183911056, 0.6798916546022343, 0.527...",48,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[4, 4, 4, 14, 14, 15, 9, 9, 9, 9, 14, 14, 14, ...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 9.1, 9.1, 9....","['Nos,Tube wells,Gross Net Area Irrigated Hect...","['Agriculture, Horticulture & Sericulture', 'A...",['Gross and Net area Irrigated under different...
3,4,Ensure inclusive and equitable quality educati...,"['ensure', 'inclusive', 'equitable', 'quality'...",ensure inclusive and equitable quality educati...,"[['education'], ['ensure', 'inclusive'], ['pro...",[[-0.15599 -0.48863 -0.095793 ... -0.7...,"[1013, 1014, 1016, 1017, 1021, 1022, 1023, 102...","[1.0000000000000002, 0.4491569151487498, 0.507...",41,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 14, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","['Male,Rural, Literates 2011 Census, Education...","['Education', 'Education', 'Education', 'Educa...","['Literates 2011 Census', 'Literates 2011 Cens..."
4,5,Achieve gender equality and empower all women ...,"['achieve', 'gender', 'equality', 'empower', '...",achieve gender equality and empower all women ...,"[['women'], ['equality', 'empower'], ['equalit...",[[-0.43895 0.47641999 0.18844 ... 0.2...,"[1034, 1038, 1042, 1055, 1058, 1063, 1073, 108...","[0.6183490921829581, 0.4247187957850606, 0.400...",46,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 2,...","[10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10.10, 10...","['Girls,Total Enrolment, No.of Lower Primary S...","['Education', 'Education', 'Education', 'Educa...",['No.of Lower Primary Schools From 1st To 5th ...
5,6,Ensure availability and sustainable management...,"['ensure', 'availability', 'sustainable', 'man...",ensure availability and sustainable management...,"[['ensure', 'availability'], ['ensure'], ['wat...",[[ 0.21560501 -0.03204 0.1458025 ... -0.0...,"[18, 1002, 1068, 1078, 1386, 1387, 1388, 1389,...","[0.5807085016601672, 0.5270143992244825, 1.000...",52,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.

In [20]:
sdg_threshold = pd.read_csv(absolute_path_sem_threshold, dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, sep=";", converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_threshold

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"['end', 'poverty', 'form', 'everywhere']",end poverty in all its forms everywhere,"[['end', 'poverty'], ['end'], ['forms', 'every...",[[ 0.004565 0.380225 0.28172 ... -0.0...,"[1366, 1367, 1368, 1369, 1370, 1371, 1372, 819...",200,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[14, 14, 14, 14, 14, 14, 14, 5, 5, 3, 3, 3, 5,...","[14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 14.1, 5.4...","['Ddu-gky,Nrlm National Rural Livelihood Missi...","['Rural Development and Panchayat Raj', 'Rural...",['Skill Development and Self Employment at the...
1,2,"End hunger, achieve food security and improved...","['end', 'hunger', 'achieve', 'food', 'security...",end hunger achieve food security and improved ...,"[['sustainable', 'agriculture'], ['security', ...",[[ 0.079023 0.0409187 -0.25506 ... -0.3...,"[380, 381, 455, 456, 510, 858, 859, 860, 861, ...",627,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[4, 4, 4, 4, 4, 6, 6, 6, 6, 17, 17, 17, 17, 17...","[4.3, 4.3, 4.6, 4.6, 4.12, 6.2, 6.2, 6.2, 6.2,...","['Total Food Grains,Pulses, Area Under Princip...","['Agriculture, Horticulture & Sericulture', 'A...",['Area Under Principal crops in Hectares 2018-...
2,3,Ensure healthy lives and promote well-being fo...,"['ensure', 'healthy', 'life', 'promote', 'well...",ensure healthy lives and promote well being fo...,"[['well'], ['well', 'ages'], ['ensure'], ['pro...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[352, 353, 354, 1393, 1394, 1414, 967, 968, 96...",236,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[4, 4, 4, 14, 14, 15, 9, 9, 9, 9, 9, 14, 14, 1...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 9.1, 9.1, 9....","['Nos,Tube wells,Gross Net Area Irrigated Hect...","['Agriculture, Horticulture & Sericulture', 'A...",['Gross and Net area Irrigated under different...
3,4,Ensure inclusive and equitable quality educati...,"['ensure', 'inclusive', 'equitable', 'quality'...",ensure inclusive and equitable quality educati...,"[['education'], ['ensure', 'inclusive'], ['pro...",[[-0.15599 -0.48863 -0.095793 ... -0.7...,"[1013, 1014, 1016, 1017, 1021, 1022, 1023, 102...",219,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","['Male,Rural, Literates 2011 Census, Education...","['Education', 'Education', 'Education', 'Educa...","['Literates 2011 Census', 'Literates 2011 Cens..."
4,5,Achieve gender equality and empower all women ...,"['achieve', 'gender', 'equality', 'empower', '...",achieve gender equality and empower all women ...,"[['women'], ['equality', 'empower'], ['equalit...",[[-0.43895 0.47641999 0.18844 ... 0.2...,"[1034, 1038, 1042, 1055, 1058, 1063, 1073, 108...",465,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10.10, 10...","['Girls,Total Enrolment, No.of Lower Primary S...","['Education', 'Education', 'Education', 'Educa...",['No.of Lower Primary Schools From 1st To 5th ...
5,6,Ensure availability and sustainable management...,"['ensure', 'availability', 'sustainable', 'man...",ensure availability and sustainable management...,"[['ensure', 'availability'], ['ensure'], ['wat...",[[ 0.21560501 -0.03204 0.1458025 ... -0.0...,"[18, 1002, 1068, 1078, 1386, 1387, 1388, 1389,...",248,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","['Working Fire Brigade Water Tankers, No. of F...","['General Information', 'Transport and Communi...","['No. of Fire Stations on 31-3-2020', 'Length ..."
6,7,"Ensure access to affordable, reliable, sustain...","['ensure', 'access', 'affordable', 'reliable',...",ensure access to affordable reliable sustainab.

In [21]:
result_sdg_new =sdg_string.copy()
result_sdg_sim_new =sdg_sim.copy()
result_sdg_threshold_new = sdg_threshold.copy()

In [22]:
result_sdg_new = result_sdg_new[['Goal No.', 'top_n_attr']]

In [23]:
result_sdg_new.head()

,Goal No.,top_n_attr
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108..."


In [24]:
result_sdg_sim_new = result_sdg_sim_new[['Goal No.', 'top_n_attr']]

In [25]:
result_sdg_sim_new.head()

,Goal No.,top_n_attr
0,1,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,..."
1,2,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ..."
2,3,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97..."
3,4,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103..."
4,5,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109..."


In [26]:
result_sdg_threshold_new = result_sdg_threshold_new[['Goal No.', 'top_n_attr']]

In [27]:
result_sdg_threshold_new.head()

,Goal No.,top_n_attr
0,1,"[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829..."
1,2,"[390, 391, 465, 466, 520, 868, 869, 870, 871, ..."
2,3,"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97..."
3,4,"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103..."
4,5,"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109..."


In [28]:
result_sdg_new.rename(columns = {'top_n_attr':'top_n_attr_A'}, inplace=True)

In [29]:
result_sdg_new['top_n_attr_B'] = result_sdg_threshold_new.loc[:,'top_n_attr']
result_sdg_new['top_n_attr_C'] = result_sdg_sim_new.loc[:,'top_n_attr']

In [30]:
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107..."
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1..."


In [31]:
# Apply the function to the merged dataframe
result_sdg_new['B_plus_C'] = result_sdg_new.apply(combine_lists_BC, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107..."
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9..."


In [32]:
# Apply the function to the merged dataframe
result_sdg_new['A_and_BC'] = result_sdg_new.apply(find_intersection_ABC, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107..."
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9..."


In [33]:
# Apply the function to the merged dataframe
result_sdg_new['A'] = result_sdg_new.apply(find_difference_A, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[]
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[]
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[]
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[]
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[]
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[]
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[]
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[]
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...",[]


In [34]:
# Apply the function to the merged dataframe
result_sdg_new['B_and_C'] = result_sdg_new.apply(find_intersection_BC, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[],"[390, 391, 465, 466, 520, 868, 869, 870, 871, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[],"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[],"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[],"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[],"[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[],"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107..."
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...",[],"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1..."


In [35]:
# Apply the function to the merged dataframe
result_sdg_new['B_minus_all'] = result_sdg_new.apply(find_difference_B, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[319, 321, 824, 826, 1392, 1393, 1394, 1395, 3..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[],"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[1153, 1154, 1155, 1156, 1157, 1158, 1159, 116..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[],"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[1344, 1345, 1346, 1347, 1348, 1349, 1350, 135..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[],"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1344, 1345, 1346, 1348, 1349, 1350, 1526, 152..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[],"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[167, 170, 173, 176, 179, 195, 199, 203, 207, ..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1014, 1015, 1016, 1017, 1359, 1365, 1367, 137..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[],"[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[1219, 1220, 1358, 1359, 1365, 1367, 1383, 138..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[],"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1347, 1358, 1359, 1360, 1361, 1362, 1363, 136..."
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...",[],"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[589, 590, 1489, 1490, 1501, 1502, 1503, 1504,..."


In [36]:
# Apply the function to the merged dataframe
result_sdg_new['C_minus_all'] = result_sdg_new.apply(find_difference_C, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[319, 321, 824, 826, 1392, 1393, 1394, 1395, 3...",[]
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[],"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[1153, 1154, 1155, 1156, 1157, 1158, 1159, 116...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33]"
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[],"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[1344, 1345, 1346, 1347, 1348, 1349, 1350, 135...",[]
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[],"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1344, 1345, 1346, 1348, 1349, 1350, 1526, 152...",[]
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[],"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[167, 170, 173, 176, 179, 195, 199, 203, 207, ...",[]
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1014, 1015, 1016, 1017, 1359, 1365, 1367, 137...",[]
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2...",[]
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[],"[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[1219, 1220, 1358, 1359, 1365, 1367, 1383, 138...",[]
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[],"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1347, 1358, 1359, 1360, 1361, 1362, 1363, 136...",[]
9,10,"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...","[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 9...",[],"[42, 46, 50, 54, 58, 62, 466, 550, 567, 809, 1...","[589, 590, 1489, 1490, 1501, 1502, 1503, 1504,...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33]"


In [37]:
result_sdg_new.columns

Index(['Goal No.', 'top_n_attr_A', 'top_n_attr_B', 'top_n_attr_C', 'B_plus_C',
       'A_and_BC', 'A', 'B_and_C', 'B_minus_all', 'C_minus_all'],
      dtype='object')

In [38]:
# Apply the function to the merged dataframe
result_sdg_new['Rank'] = result_sdg_new.apply(final_rank, axis=1)
result_sdg_new

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[319, 321, 824, 826, 1392, 1393, 1394, 1395, 3...",[],"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,..."
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[],"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[1153, 1154, 1155, 1156, 1157, 1158, 1159, 116...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33]","[390, 391, 465, 466, 520, 334, 335, 336, 337, ..."
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[],"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[1344, 1345, 1346, 1347, 1348, 1349, 1350, 135...",[],"[163, 164, 165, 166, 167, 168, 169, 170, 171, ..."
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[],"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1344, 1345, 1346, 1348, 1349, 1350, 1526, 152...",[],"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103..."
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[],"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[167, 170, 173, 176, 179, 195, 199, 203, 207, ...",[],"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108..."
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1014, 1015, 1016, 1017, 1359, 1365, 1367, 137...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,..."
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141..."
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[],"[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[1219, 1220, 1358, 1359, 1365, 1367, 1383, 138...",[],"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ..."
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[],"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1347, 1358, 1359, 1360, 1361, 1362, 1363, 136...",[],"[1071, 1072, 1073, 107

In [39]:
final_df = result_sdg_new.copy()

In [40]:
final_df['Attr_count'] = final_df['Rank'].apply(lambda x: len(x))

In [41]:
final_df

,Goal No.,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank,Attr_count
0,1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 24,...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1376, 1377, 1378, 1379, 1380, 1381, 1382, 829...","[319, 321, 824, 826, 1392, 1393, 1394, 1395, 3...",[],"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 1376,...",436
1,2,"[390, 391, 465, 466, 520, 334, 335, 336, 337, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",[],"[390, 391, 465, 466, 520, 868, 869, 870, 871, ...","[1153, 1154, 1155, 1156, 1157, 1158, 1159, 116...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33]","[390, 391, 465, 466, 520, 334, 335, 336, 337, ...",637
2,3,"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",[],"[362, 363, 364, 1403, 1404, 1424, 977, 978, 97...","[1344, 1345, 1346, 1347, 1348, 1349, 1350, 135...",[],"[163, 164, 165, 166, 167, 168, 169, 170, 171, ...",236
3,4,"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",[],"[1023, 1024, 1026, 1027, 1031, 1032, 1033, 103...","[1344, 1345, 1346, 1348, 1349, 1350, 1526, 152...",[],"[1023, 1024, 1025, 1026, 1027, 1028, 1029, 103...",219
4,5,"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",[],"[1044, 1048, 1052, 1065, 1068, 1073, 1083, 109...","[167, 170, 173, 176, 179, 195, 199, 203, 207, ...",[],"[1365, 1044, 1048, 1052, 1065, 1068, 1073, 108...",465
5,6,"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...","[1014, 1015, 1016, 1017, 1359, 1365, 1367, 137...",[],"[21, 1012, 1078, 1088, 1396, 1397, 1398, 1399,...",248
6,7,"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...","[15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2...",[],"[1011, 1012, 1013, 1014, 1015, 1016, 1017, 141...",311
7,8,"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",[],"[913, 905, 1376, 1377, 1378, 1379, 1380, 1381,...","[1219, 1220, 1358, 1359, 1365, 1367, 1383, 138...",[],"[829, 905, 913, 1376, 1377, 1378, 1379, 1380, ...",126
8,9,"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...",[],"[1071, 1072, 1073, 1074, 1075, 1076, 1077, 107...","[1347, 1358, 1359, 1360, 1361, 136

In [42]:
final_df.to_csv(absolute_path_rank, sep=';', index=False)

In [43]:
all_goals = process_result(final_df, sdg, data)

In [44]:
all_goals.to_csv(absolute_path_rank_unroll, sep=';', index=False)

In [45]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
4564,1134,10,Education,10.13,No.of Medical Colleges and Students 2019-20,"Total,Private,Dental Colleges","Data, monitoring and accountability",17
4565,1140,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
4566,1141,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Girls,Students from 1st year to final year,Pri...","Data, monitoring and accountability",17
4567,1148,10,Education,10.15,No.of Engineering Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17


In [46]:
# Group by 'col1' and count the distinct values in 'col2'
table_map = data_new.groupby('Chapter_id')['Table_id'].nunique().to_dict()
table_map

{'1': 10,
 '10': 15,
 '11': 6,
 '12': 6,
 '13': 6,
 '14': 5,
 '15': 9,
 '16': 3,
 '17': 3,
 '2': 21,
 '3': 2,
 '4': 19,
 '5': 4,
 '6': 4,
 '7': 2,
 '8': 4,
 '9': 4}

In [47]:
attr_map = data_new.groupby('Table_id')['Attr_id'].nunique().to_dict()
attr_map

{'1.1': 4,
 '1.10': 7,
 '1.2': 3,
 '1.3': 6,
 '1.4': 4,
 '1.5': 4,
 '1.6': 15,
 '1.7': 8,
 '1.8': 8,
 '1.9': 8,
 '10.1': 9,
 '10.10': 10,
 '10.11': 11,
 '10.12': 9,
 '10.13': 24,
 '10.14': 8,
 '10.15': 8,
 '10.2': 10,
 '10.3': 4,
 '10.4': 4,
 '10.5': 4,
 '10.6': 5,
 '10.7': 5,
 '10.8': 7,
 '10.9': 10,
 '11.1': 14,
 '11.2': 16,
 '11.3': 7,
 '11.4': 5,
 '11.5': 23,
 '11.6': 8,
 '12.1': 24,
 '12.2': 18,
 '12.3': 30,
 '12.4': 6,
 '12.5': 18,
 '12.6': 21,
 '13.1': 6,
 '13.2': 4,
 '13.3': 5,
 '13.4': 5,
 '13.5': 8,
 '13.6': 7,
 '14.1': 7,
 '14.2': 6,
 '14.3': 7,
 '14.4': 7,
 '14.5': 6,
 '15.1': 7,
 '15.2': 6,
 '15.3': 9,
 '15.4': 8,
 '15.5': 5,
 '15.6': 9,
 '15.7': 10,
 '15.8': 10,
 '15.9': 5,
 '16.1': 7,
 '16.2': 16,
 '16.3': 20,
 '17.1': 4,
 '17.2': 19,
 '17.3': 19,
 '2.1': 18,
 '2.11': 20,
 '2.12': 11,
 '2.13': 34,
 '2.14': 10,
 '2.15': 10,
 '2.16': 10,
 '2.17': 10,
 '2.18': 10,
 '2.19': 10,
 '2.2': 10,
 '2.20': 10,
 '2.21': 10,
 '2.22': 10,
 '2.3': 7,
 '2.4': 12,
 '2.5': 7,
 '2.6': 10,
 

In [48]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
4564,1134,10,Education,10.13,No.of Medical Colleges and Students 2019-20,"Total,Private,Dental Colleges","Data, monitoring and accountability",17
4565,1140,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
4566,1141,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Girls,Students from 1st year to final year,Pri...","Data, monitoring and accountability",17
4567,1148,10,Education,10.15,No.of Engineering Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17


In [49]:
filter_df = all_goals.copy()

In [50]:
# Initialize an empty dictionary to store the counts
tab_result = {}
table_id_rm = {}
# Iterate over unique values in col1
for val in filter_df['top_n_goal_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = filter_df[filter_df['top_n_goal_id'] == val]
    
    # Count the occurrences of each value in col2 and store in a dictionary
    table_counts = filtered_df['Table_id'].value_counts().to_dict()
    
    table_id_rm[val] = {key:value for key, value in table_counts.items() if value < (attr_map[key]/2)}
    
    table_dict_new = {key:value for key, value in table_counts.items() if key not in table_id_rm[val]}
    
    if(len(table_dict_new) <= 4):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(table_counts.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_6_dict = dict(list(sorted_dict.items())[:6])
        tab_result[val] = top_6_dict
        
    else:
        # Add the col2 counts dictionary to the result dictionary
        tab_result[val] = table_dict_new

print(tab_result)

{'1': {'4.19': 197, '12.3': 30, '12.1': 24, '12.6': 21, '4.2': 20, '17.2': 19, '12.5': 18, '12.2': 18, '1.6': 15, '11.6': 8, '1.9': 8, '1.8': 8, '1.7': 8, '14.1': 7, '12.4': 6, '3.1': 5, '14.3': 4}, '2': {'4.19': 197, '4.4': 47, '11.5': 23, '4.18': 22, '4.3': 20, '4.2': 20, '4.17': 20, '4.1': 19, '17.2': 19, '4.6': 19, '4.12': 18, '11.2': 16, '11.1': 14, '4.15': 13, '1.6': 10, '4.10': 9, '4.13': 9, '4.11': 9, '4.7': 9, '13.5': 8, '4.5': 8, '11.6': 8, '14.3': 7, '14.4': 7, '14.1': 7, '13.6': 7, '11.3': 7, '14.2': 6, '4.9': 6, '14.5': 6, '13.1': 6, '13.4': 5, '13.3': 5, '4.16': 5, '11.4': 5, '13.2': 4, '4.14': 3, '4.8': 3}, '3': {'11.5': 23, '2.11': 20, '17.2': 19, '11.2': 16, '11.1': 14, '1.6': 9, '1.8': 8, '11.6': 8, '13.5': 8, '1.9': 8, '14.1': 7, '14.3': 7, '10.8': 7, '11.3': 7, '14.4': 7, '13.6': 7, '14.5': 6, '14.2': 6, '13.1': 6, '13.4': 5, '13.3': 5, '11.4': 5, '3.1': 5, '13.2': 4}, '4': {'10.13': 24, '17.2': 19, '10.11': 11, '10.2': 10, '8.4': 10, '10.9': 10, '10.10': 10, '10.1'

In [51]:
len(tab_result)

17

In [52]:
table_id_rm

{'1': {'11.5': 6,
  '9.1': 5,
  '5.4': 4,
  '14.5': 2,
  '4.11': 1,
  '6.3': 1,
  '15.3': 1},
 '2': {'6.2': 4, '7.2': 3, '5.3': 1, '8.2': 1, '7.1': 1, '8.1': 1},
 '3': {'4.18': 8, '9.1': 5, '4.2': 3, '5.4': 2, '15.3': 1},
 '4': {'9.1': 6, '6.2': 1},
 '5': {'4.19': 48,
  '6.2': 13,
  '2.13': 9,
  '17.3': 8,
  '11.5': 7,
  '6.1': 7,
  '2.11': 6,
  '4.18': 4,
  '2.22': 3,
  '2.16': 3,
  '2.21': 3,
  '2.20': 3,
  '2.19': 3,
  '2.18': 3,
  '2.17': 3,
  '2.15': 3,
  '2.14': 3,
  '2.6': 2,
  '2.2': 2,
  '2.1': 2,
  '2.8': 2,
  '1.4': 1},
 '6': {'4.18': 8, '4.2': 3, '16.3': 2, '16.2': 2, '1.5': 1, '5.4': 1},
 '7': {'4.18': 8, '4.1': 4, '8.4': 2, '9.4': 1},
 '8': {'4.18': 9,
  '9.1': 5,
  '4.17': 3,
  '4.2': 3,
  '11.6': 2,
  '16.2': 2,
  '5.4': 2,
  '15.3': 1,
  '6.2': 1,
  '3.1': 1,
  '1.5': 1},
 '9': {'4.18': 8},
 '10': {'4.18': 3,
  '16.2': 2,
  '1.7': 2,
  '1.8': 2,
  '11.2': 2,
  '6.2': 2,
  '1.9': 2,
  '10.12': 1,
  '9.2': 1,
  '9.1': 1,
  '5.1': 1,
  '4.17': 1,
  '4.16': 1,
  '4.6': 1,


In [53]:
sum_ind=0
# Iterate over the outer dictionary
for key, inner_dict in table_id_rm.items():
    # Iterate over the inner dictionary
    for k, value in inner_dict.items():
        if k not in tab_result[key]:
            # Add the value to the result
            sum_ind += value

print(sum_ind)

459


In [54]:
# List to store indices to remove
indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in filter_df.iterrows():
    if row['top_n_goal_id'] in table_id_rm:
        inner_dict = table_id_rm[row['top_n_goal_id']]
        if row['Table_id'] in inner_dict:
            if row['Table_id'] not in tab_result[row['top_n_goal_id']]:
                # Add index to remove list
                indices_to_remove.append(index)
            
indices_to_remove

[250,
 251,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 275,
 276,
 283,
 284,
 285,
 286,
 287,
 288,
 305,
 435,
 892,
 893,
 894,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 936,
 1093,
 1094,
 1095,
 1105,
 1109,
 1110,
 1111,
 1112,
 1113,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1221,
 1222,
 1447,
 1508,
 1523,
 1524,
 1525,
 1526,
 1527,
 1624,
 1625,
 1626,
 1627,
 1628,
 1629,
 1630,
 1631,
 1632,
 1633,
 1654,
 1655,
 1656,
 1660,
 1661,
 1662,
 1663,
 1664,
 1665,
 1666,
 1667,
 1668,
 1669,
 1670,
 1671,
 1672,
 1673,
 1674,
 1675,
 1676,
 1677,
 1678,
 1679,
 1680,
 1681,
 1682,
 1683,
 1684,
 1685,
 1686,
 1687,
 1688,
 1689,
 1690,
 1691,
 1692,
 1693,
 1694,
 1695,
 1696,
 1697,
 1698,
 1699,
 1700,
 1701,
 1702,
 1703,
 1704,
 1705,
 1706,
 1707,
 1708,
 1709,
 1710,
 1711,
 1712,
 1713,
 1714,
 1715,
 1716,
 1717,
 1718,
 1719,
 1720,
 1721,
 1722,
 1723,
 1724,
 1725,
 1726,
 1727,
 1728,
 1729,
 1730,
 1731,
 1732,
 1733,
 1734,
 1735,
 1736,
 1

In [55]:
len(indices_to_remove)

459

In [56]:
# Remove rows using indices from indices_to_remove list
filter_df.drop(indices_to_remove, inplace=True)

filter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
4564,1134,10,Education,10.13,No.of Medical Colleges and Students 2019-20,"Total,Private,Dental Colleges","Data, monitoring and accountability",17
4565,1140,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
4566,1141,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Girls,Students from 1st year to final year,Pri...","Data, monitoring and accountability",17
4567,1148,10,Education,10.15,No.of Engineering Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17


In [57]:
filter_df.to_csv(absolute_path_tab_filter, sep=';', index=False)

In [58]:
tab_count_att = filter_df['top_n_goal_id'].value_counts().to_dict()

In [59]:
filter_df.to_csv(absolute_path_final_map, sep=';', index=False)

In [60]:
#filter_df_copy = filter_df.copy()

In [61]:
def filter_dataframe(df, n, chap_list):
    result_df = df.copy() # Initialize an empty DataFrame to store the filtered rows
    
    # Loop over each unique 'Target_id'
    for target_id, group in df.groupby('top_n_goal_id'):
        num_rows = len(group)  # Count the number of rows for this 'Target_id'
        
        # Check if the number of rows is greater than the threshold 'n'
        if num_rows > n:
            idx_list = []
            drop_list = []
            for chap in chap_list:
                # Find the indices corresponding to chapters "2" and "3" for this 'Target_id'
                idx_chapter = group[group['Chapter_id'] == chap].index
                idx_list.append(idx_chapter)
                
            for item in idx_list:
                if len(item)>1:
                    if len(item)>300:
                        total_rows_to_drop = item[len(item)//3:]
                    else:
                        total_rows_to_drop = item[len(item)//2:]
                    drop_list.append(total_rows_to_drop)
            
            for i in drop_list:
                if len(i)>0:
                    result_df = result_df.drop(i)
    result_df.reset_index(inplace = True, drop = True)
    return result_df

In [62]:
#chap_ll = ['2', '4']
# Filter the DataFrame using the function
#final_map = filter_dataframe(filter_df_copy, 350, chap_ll)

In [63]:
#final_map.to_csv(absolute_path_final_map, sep=';', index=False)

In [64]:
chapter_df = filter_df.copy()

In [65]:
# Initialize an empty dictionary to store the counts
chap_result = {}
chap_id_rm = {}

# Iterate over unique values in col1
for val in chapter_df['top_n_goal_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = chapter_df[chapter_df['top_n_goal_id'] == val]
    chap_list = []
    for tab in filtered_df['Table_id'].unique():
        chap_filter = filtered_df[filtered_df['Table_id'] == tab]
        
        # Count the occurrences of each value in col2 and store in a dictionary
        chap_counts = chap_filter['Chapter_id'].unique().tolist()
        
        chap_list.extend(chap_counts)
    chap_dict = pd.Series(chap_list).value_counts().to_dict()
    chap_id_rm[val] = {key:value for key, value in chap_dict.items() if value < (table_map[key]/2)}
    chap_dict_new = {key:value for key, value in chap_dict.items() if key not in chap_id_rm[val]}
    if(len(chap_dict_new) <= 1):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(chap_dict.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_4_dict = dict(list(sorted_dict.items())[:4])
        chap_result[val] = top_4_dict
 
    else:
        # Add the col2 counts dictionary to the result dictionary
        chap_result[val] = chap_dict_new

print(chap_result)

{'1': {'12': 6, '3': 1}, '2': {'4': 19, '13': 6, '11': 6, '14': 5}, '3': {'13': 6, '11': 6, '14': 5, '3': 1}, '4': {'10': 15, '1': 3, '14': 2, '13': 2}, '5': {'10': 15, '13': 6, '12': 6}, '6': {'13': 6, '11': 6, '14': 5}, '7': {'1': 10, '15': 9, '13': 6, '14': 5, '16': 3, '17': 3}, '8': {'13': 6, '14': 5}, '9': {'13': 6, '14': 5}, '10': {'12': 6, '17': 2}, '11': {'13': 6, '14': 5}, '12': {'13': 6, '14': 5, '17': 2}, '13': {'2': 5, '1': 4, '8': 4, '13': 2}, '14': {'13': 6, '14': 5}, '15': {'1': 10, '13': 6, '14': 5, '17': 3, '12': 3}, '16': {'13': 6, '14': 5, '6': 4}, '17': {'10': 15, '17': 1, '11': 1}}


In [66]:
chap_id_rm

{'1': {'1': 4, '14': 2, '4': 2, '17': 1, '11': 1},
 '2': {'17': 1, '1': 1},
 '3': {'1': 3, '2': 1, '10': 1, '17': 1},
 '4': {'1': 3, '14': 2, '13': 2, '8': 1, '17': 1, '9': 1},
 '5': {'14': 2, '2': 2, '17': 1},
 '6': {'1': 3, '10': 2, '9': 1, '15': 1, '8': 1, '17': 1},
 '7': {'9': 1},
 '8': {'6': 1, '16': 1, '17': 1},
 '9': {'10': 2},
 '10': {'13': 2, '16': 1, '1': 1},
 '11': {'4': 1, '11': 1},
 '12': {'4': 7, '1': 3, '5': 1, '16': 1, '11': 1},
 '13': {'2': 5, '1': 4, '13': 2, '15': 1},
 '14': {'4': 3, '2': 3, '15': 1, '1': 1},
 '15': {'10': 5, '9': 1, '8': 1},
 '16': {'2': 4, '4': 1, '8': 1},
 '17': {'17': 1, '11': 1}}

In [67]:
missing = []

# Iterate through the data dictionary
for key, value in chap_result.items():
    # Check if the sub-dictionary is empty
    if not value:
        # Add the key to the missing list
        missing.append(key)
missing

[]

In [68]:
# List to store indices to remove
chap_indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in chapter_df.iterrows():
    if row['top_n_goal_id'] in table_id_rm:
        inner_dict = chap_id_rm[row['top_n_goal_id']]
        if row['Chapter_id'] in inner_dict:
            if row['top_n_goal_id'] not in chap_result or row['Chapter_id'] not in chap_result[row['top_n_goal_id']]:
                # Add index to remove list
                chap_indices_to_remove.append(index)

chap_indices_to_remove

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [69]:
len(chap_indices_to_remove)

1202

In [70]:
# Remove rows using indices from indices_to_remove list
chapter_df.drop(chap_indices_to_remove, inplace=True)

chapter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
252,318,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Working,Raingauge Stations",End poverty in all its forms everywhere,1
273,319,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Under Repair,Raingauge Stations",End poverty in all its forms everywhere,1
274,321,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Rainy Days,Annual Normal Rainfall 1961-2010",End poverty in all its forms everywhere,1
281,320,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Total,Raingauge Stations",End poverty in all its forms everywhere,1
282,322,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Normal Rainfall,Annual Normal Rainfall 1961-2010",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
4564,1134,10,Education,10.13,No.of Medical Colleges and Students 2019-20,"Total,Private,Dental Colleges","Data, monitoring and accountability",17
4565,1140,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17
4566,1141,10,Education,10.14,No.of Polytechnic Colleges and Students 2019-20,"Girls,Students from 1st year to final year,Pri...","Data, monitoring and accountability",17
4567,1148,10,Education,10.15,No.of Engineering Colleges and Students 2019-20,"Boys,Students from 1st year to final year,Priv...","Data, monitoring and accountability",17


In [71]:
chapter_df['top_n_goal_id'].nunique()

17

In [72]:
chapter_df[chapter_df['top_n_goal_id']=='3']

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
1103,1403,14,Rural Development and Panchayat Raj,14.5,Drinking Water Supply Facilities on 31-03-2020,"Borewells drilled in 2019-20,Bore Wells drilled",Ensure healthy lives and promote well-being fo...,3
1104,1404,14,Rural Development and Panchayat Raj,14.5,Drinking Water Supply Facilities on 31-03-2020,"Total Borewells on 31-3-2020,Bore Wells drilled",Ensure healthy lives and promote well-being fo...,3
1114,1389,14,Rural Development and Panchayat Raj,14.3,Phisical Progress under Mahathma Gandhi Nation...,No. of Registered and Distributed Job Cards,Ensure healthy lives and promote well-being fo...,3
1115,1390,14,Rural Development and Panchayat Raj,14.3,Phisical Progress under Mahathma Gandhi Nation...,No. of Householdsdemanded Employment,Ensure healthy lives and promote well-being fo...,3
1116,1391,14,Rural Development and Panchayat Raj,14.3,Phisical Progress under Mahathma Gandhi Nation...,Number of Households Provided employement,Ensure healthy lives and promote well-being fo...,3
...,...,...,...,...,...,...,...,...
1279,1219,11,Health & Family Welfare Services,11.6,Different Health Facilities on 31-01-2020,"Nos,Number of working Hospitals",Ensure healthy lives and promote well-being fo...,3
1280,1220,11,Health & Family Welfare Services,11.6,Different Health Facilities on 31-01-2020,Delivaries occured in Hospitals during 2019-20...,Ensure healthy lives and promote well-being fo...,3
1281,1221,11,Health & Family Welfare Services,11.6,Different Health Facilities on 31-01-2020,No. of Beneficiaries availed Janani Suraksha Y...,Ensure healthy lives and promote well-being fo...,3
1282,1222,11,Health & Family Welfare Services,11.6,Different Health Facilities on 31-01-2020,No. of Beneficiaries received Madilu kit durin...,Ensure healthy lives and promote well-being fo...,3


In [73]:
chap_count_att = chapter_df['top_n_goal_id'].value_counts().to_dict()

In [74]:
chap_count_att

{'2': 597,
 '7': 289,
 '5': 280,
 '15': 213,
 '4': 176,
 '17': 170,
 '16': 151,
 '3': 146,
 '6': 141,
 '10': 140,
 '1': 122,
 '13': 120,
 '12': 91,
 '14': 68,
 '11': 68,
 '8': 68,
 '9': 68}

In [75]:
tab_count_att

{'2': 626,
 '1': 416,
 '14': 339,
 '5': 327,
 '7': 296,
 '15': 253,
 '6': 231,
 '12': 224,
 '3': 217,
 '16': 213,
 '4': 212,
 '10': 179,
 '17': 170,
 '13': 127,
 '11': 96,
 '8': 96,
 '9': 88}

In [76]:
rank_count_att = all_goals['top_n_goal_id'].value_counts().to_dict()

In [77]:
#rank_filter_count_att = final_map['top_n_goal_id'].value_counts().to_dict()

In [78]:
count_att_list = [(key, value) for key, value in chap_count_att.items()]

count_att_df = pd.DataFrame(count_att_list, columns=['Goal No.', 'Chap_count'])
count_att_df

,Goal No.,Chap_count
0,2,597
1,7,289
2,5,280
3,15,213
4,4,176
5,17,170
6,16,151
7,3,146
8,6,141
9,10,140


In [79]:
count_att_df['Tab_count'] = count_att_df['Goal No.'].map(tab_count_att)

In [80]:
#count_att_df['Filter_count'] = count_att_df['Goal No.'].map(rank_filter_count_att)

In [81]:
count_att_df['Rank_count'] = count_att_df['Goal No.'].map(rank_count_att)

In [82]:
count_att_df

,Goal No.,Chap_count,Tab_count,Rank_count
0,2,597,626,637
1,7,289,296,311
2,5,280,327,465
3,15,213,253,341
4,4,176,212,219
5,17,170,170,170
6,16,151,213,222
7,3,146,217,236
8,6,141,231,248
9,10,140,179,202


In [83]:
chapter_df.to_csv(absolute_path_chap_filter, sep=';', index=False)

In [84]:
count_att_df.to_csv(absolute_path_count, sep=';', index=False)